In [45]:
import pandas as pd 
import pickle
import nltk
import numpy as np
from nltk.collocations import *
import spacy
from modules.preprocess_articles import *
from modules.collocation_articles import *
from tqdm import tqdm
import json # use json instead of pickle for persistence

In [46]:
# to load the corpus
with open("data/factiva_data.json", 'r') as f:
    factiva_corpus = json.load(f)

In [47]:
# Load model
spacy_mod = spacy.load("de_core_news_lg",
                 disable=['ner', 'parser', 'tagger'])

In [48]:
# select first 100 articles for demo
## NB: perhaps better to select relevant articles instead?
for i, item in enumerate(factiva_corpus):
    factiva_corpus[i] = item[:100]

In [49]:
print(factiva_corpus[1]) # prints lowercase correctly

["['es gibt physische gewalt, es gibt psychische gewalt - und es gibt digitale gewalt. wie diese wirkt, wenn sie vom staat als waffe eingesetzt wird, das erklärten am samstag in berlin dissidenten und journalisten, die opfer der spähsoftware pegasus geworden sind: „es ist, als säße jemand in deinem gehirn“, sagte der anwalt mazen masri. „unsere privatsphäre wurde zerstört“, beschrieb es die mexikanische journalistin carmen aristegui. und der nsa-whistleblower edward snowden, per video zugeschaltet, warnte eindringlich vor der drohenden „ausspäh-gesellschaft“ und vor der erosion der demokratie.', 'nicht nur aristegui und ihren kollegen, auch ihrem minderjährigen sohn wurde pegasus von staatlichen organen aufs handy gespielt. die software erlaubt nicht nur, unbemerkt sämtliche daten des opfers abzugreifen und dessen gesamte kommunikation zu verfolgen, sondern auch, diesen mit kamera und mikrofon seines eigenen geräts zu überwachen, sogar unter seinem namen anrufe zu tätigen und nachricht

In [50]:
# Preprocessing pipeline 
newspaper_preprocessor = Preprocessor(newspaper_data = factiva_corpus, nlp = spacy_mod)
newspaper_preprocessor.tokenize()
newspaper_preprocessor.preprocess()

# Individual articles 
factiva_preprocessed = newspaper_preprocessor.return_preprocessed()

# Joint articles
## NB: What is the difference between individual and joint articles?
factiva_preprocessed_joint = [j for i in factiva_preprocessed for j in i]

In [51]:
# pretty print tokens for individual articles
for ls in factiva_preprocessed:
    print(*ls)

physisch Gewalt psychisch Gewalt digital Gewalt wirken Staat Waffe einsetzen erklären Samstag Berlin Dissident Journalist Opfer spähsoftwar Pegasus säße Gehirn Anwalt maz Masri Privatsphäre zerstören beschreiben mexikanisch Journalistin carmen Aristegui Edward snowden per Video zuschalten warnen eindringlich drohend Erosion Demokratie Aristegui Kollege minderjährig Sohn Pegasus staatlich Organ auf handy spielen Software erlauben unbemerkt sämtlicher daten Opfer abgreifen gesamt Kommunikation verfolgen Kamera mikrofon Gerät überwachen sogar Name anrufe tätigen Nachricht verschicken links Netzwerk betroffen infizieren Pegasus israelisch Softwarefirma nso Group entwickeln Staat Mexiko vereinigt arabisch emirat Spanien verkaufen Software schützen Welt Terrorist beteuern Firma autokratisch Herrscher nützen öfter Dissident Aktivist Journalist anzugreifen einschüchtern Journalist kritisch nso Group Firmengeflecht eingesponn berichten Opfer einzelfälle Bericht Untersuchung kanadisch Forscherte

In [22]:
# Collocations inside individual articles
articles_collocations = []

for doc in tqdm(factiva_preprocessed): 
    collocations = CollocationArticles(doc, nltk.collocations)
    articles_collocations.append(collocations.BigramCollocations(doc_type = False))

# Collocations across documents 
doc_finder = CollocationArticles(factiva_preprocessed, nltk.collocations)
docs_collocations = doc_finder.BigramCollocations(doc_type = True)

# Collocations in joint articles 
joint_finder = CollocationArticles(factiva_preprocessed_joint, nltk.collocations)
articles_collocations_joint = joint_finder.BigramCollocations(doc_type = False)

# try this: https://stackoverflow.com/questions/41094134/finding-top-bigrams-across-multiple-large-files
# inspo: https://github.com/nicharuc/Collocations/blob/master/Collocations.ipynb
# documentation: https://www.nltk.org/howto/collocations.html


100%|████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1568.24it/s]


In [23]:
docs_collocations 
# NB: only gives you top 10 across all docs without rawfreq/PMI

[('häuslich', 'Gewalt'),
 ('visa', 'vie'),
 ('Nicole', 'Jäger'),
 ('Million', 'Euro'),
 ('Nadira', 'Khalikova'),
 ('Text', 'us'),
 ('Forensic', 'Architecture'),
 ('Fall', 'häuslich'),
 ('Opfer', 'häuslich'),
 ('Fassung', 'Redaktion')]

### Testing preprocessing pipeline

In [60]:
# simple test for Preprocessor class 
# run test_processor.py now and after fixing the pipeline

# create some plain text data
text = ["th3s! `is s^Om3 SAMPle text9",
        "i'm TryiNG out something Here", 
        "Let's u£s se*e IF it's works31"]

In [62]:
# run through preprocessing pipeline 
text_preprocessor = Preprocessor(newspaper_data = text, nlp = spacy_mod)
text_preprocessor.tokenize()
text_preprocessor.preprocess()

text_preprocessed = text_preprocessor.return_preprocessed()
print(text_preprocessed) # should print clean tokens but fails

# NOTES:
# processing pipeline is not reproducible + not robust 
# because it cannot be replicated for a simpler use case
# instead of constructing Preprocesser class,
# use simple functions for each preprocessing step

IndexError: list index out of range